In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [48]:
url_inicial = 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive/2'
url_root = 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive/2'
r=requests.get(url_inicial)

In [49]:
soup = BeautifulSoup(r.text, 'html.parser')

In [50]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/57',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/55',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/54',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/64',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/53',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/49',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/50',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/51',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/52',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/45',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/46',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/47',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/48',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/41',
 'http

In [51]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

AttributeError: 'NoneType' object has no attribute 'findAll'

In [ ]:
vol2

In [40]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [41]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive


In [42]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

47

In [43]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [44]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Veterinaria México OA'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    tem='Ciencias Veterinarias y de la Producción Animal'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [45]:
list_scraper=list_scraper[0:47]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [46]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Economic analysis of different pregnancy rates...,None,The pregnancy rate (PR) is nowadays the most i...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
1,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Movement and feeding habits of semi-confined d...,None,Free-range pigs are in the highest risk to acq...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
2,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Field immobilization of Desert Bobcats (Lynx r...,None,Chemical immobilization is used to minimize st...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
3,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Quincuagésimo Aniversario,None,None,https://veterinariamexico.fmvz.unam.mx/index.p...,None
4,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,50 Aniversario,None,None,https://veterinariamexico.fmvz.unam.mx/index.p...,None
5,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Blood cell morphometry of wild Gopherus flavom...,None,Morphometric characteristics of blood cells we...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
6,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Detection of mammary adenocarcinoma metastases...,None,Mammary adenocarcinomas with metastases are mo...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
7,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,"Use of medetomidine, midazolam, ketamine and s...",None,Veterinaria México OAISSN: 2448-6760Cite this ...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
8,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Assessment of Turkish consumer attitudes using...,None,Veterinaria México OAISSN: 2448-6760Cite this ...,https://veterinariamexico.fmvz.unam.mx/index.p...,None
9,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Phenotypic variation in American bullfrogs (Li...,None,Veterinaria México OAISSN: 2448-6760Cite this ...,https://veterinariamexico.fmvz.unam.mx/index.p...,None


In [47]:
df_catalogo.to_csv('Revista57.02.csv', index=False)